# Testing on GSM8k Math reasoning

In this notebook, we show how Ally can learn and improve math reasoning skills based on [GSM8k dataset](https://github.com/openai/grade-school-math).

We use a handful of annotated examples from train set, and verify the answer accuracy on the full test set that consists of 1319 examples.

In [1]:
import sys
sys.path.append('../')

import pandas as pd
from datasets import load_dataset
gsm8k = load_dataset("gsm8k", "main")

df_train = pd.DataFrame({'question': gsm8k['train']['question'], 'answer': gsm8k['train']['answer']})
df_test = pd.DataFrame({'question': gsm8k['test']['question'], 'answer': gsm8k['test']['answer']})

df_train.head()

/Users/nguyenduyanh/miniforge3/envs/ally/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


The following code is used to evaluate the quality of answers by comparing numbers in the outputs:

In [2]:
import re

def extract_and_convert_numbers(text):
  pattern = "\d{1,3}(?:,\d{3})*\.?\d*"
  numbers = re.findall(pattern, text)
  return [int(num.replace(',', '').split('.')[0]) for num in numbers if num.replace(',', '').split('.')[0]]

def evaluate_answers(ground_truth, prediction):
  pred = extract_and_convert_numbers(prediction)
  gt = extract_and_convert_numbers(ground_truth)                             
  return any(p == gt[-1] for p in pred)

Now we can create and execute baseline agent. We start with the naive baseline that just expects answer given the question, by following the prompt template, without any additional instructions:
```
Q: {question}
A: {answer}
```

In [3]:
from ally.skills.skillset import LinearSkillSet
from ally.skills.base import TransformSkill

skills = LinearSkillSet(skills=[
	TransformSkill(
		name='math_solver',
		# we start with no instructions then explain how agent can learn more details
		instructions='',
		# instructions=prompt,
		input_template='Q: {question}',
		# here is the baseline established in Kojima et al., 2022 paper
		# output_template='A: Let’s think step by step. {rationale}\nFinal numerical answer:{answer}',
		output_template=[{
      "name": "answer",
      "description": "The final numerical answer",
		}]
	)
])

In [4]:
from ally.runtimes.openai import OpenAIRuntime
from app.core.settings import settings

openai_runtime = OpenAIRuntime(
  verbose=True,
  api_key=settings.openai_api_key,
  gpt_model_name="gpt-3.5-turbo",
)
openai_teacher_runtime = OpenAIRuntime(
  verbose=True,
  api_key=settings.openai_api_key,
  gpt_model_name="gpt-4",
)

In [5]:
from ally.agents.base import Agent
from ally.environments.base import StaticEnvironment


agent = Agent(
	skills=skills,
  runtimes={
    'openai': openai_runtime
	},
  teacher_runtimes={
    'openai': openai_teacher_runtime
	},
	environment=StaticEnvironment(
		df=df_train,
	)
)

In [6]:
# run baseline agent on a test dataset
result_baseline = agent.run(df_test.drop(columns='answer')[:5])

Applying skill: math_solver

  0%|          | 0/5 [00:00<?, ?it/s]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for 
her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. 
How much in dollars does she make every day at the farmers' market?"}

 40%|████      | 2/5 [00:00<00:00,  3.00it/s]

{'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it 
take?'}

...

 80%|████████  | 4/5 [00:02<00:00,  1.56it/s]

{'question': 'James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters
does he run a week?'}

100%|██████████| 5/5 [00:03<00:00,  1.35it/s]

{'question': "Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, 
mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In 
the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 
cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of
Wendi's flock is 20 chickens?"}

100%|██████████| 5/5 [00:03<00:00,  1.26it/s]


In [7]:
# evaluate baseline results
accuracy = StaticEnvironment(df=df_test, matching_function=evaluate_answers).get_feedback(skills, result_baseline).get_accuracy()

print(f'Baseline accuracy: {accuracy["answer"]}')

Baseline accuracy: 0.4


# Improve the baseline

The agent can improve its initial skill by learning new instructions from provided ground truth signals:

In [8]:
agent.learn(batch_size=5, learning_iterations=5)

=> Iteration #0: Getting feedback, analyzing and improving ...

Applying skill: math_solver

  0%|          | 0/5 [00:00<?, ?it/s]

{'question': 'When three friends sold their video games, Ryan, the first of the three friends, received $50 more 
than Jason from his sales. Jason received 30% more money than Zachary from selling his video games. If Zachary sold
40 games at $5 each, calculate the total amount of money the three friends received together from the sale of the 
video games.', 'answer': 'The total amount of money that Zachary received from selling his video games is $40*5 = 
$<<40*5=200>>200\nJason received 30/100*$200 = $<<30/100*200=60>>60 more than the amount Zachary received after 
selling his games.\nThe total amount of money that Jason received is $60+$200 = $<<60+200=260>>260\nTogether, 
Zachary and Jason received $260+$200 = $<<260+200=460>>460 from the sale of their video games.\nRyan received 
$260+$50=$<<260+50=310>>310 from the sale of his video games.\nThe three friends made a total of $310+$460 = 
$<<310+460=770>>770 from the sale of the video games.\n#### 770'}

 40%|████      | 2/5 [00:06<00:09,  3.06s/it]

...

{'input': '\n\t\t\tA prompt is a text paragraph that outlines the expected actions and instructs the large language
model (LLM) to \t\t\tgenerate a specific output. This prompt is concatenated with the input text, and the 
\t\t\tmodel then creates the required output.\n\t\t\tThis describes the full template how the prompt is 
concatenated with the input to produce the 
output:\n\t\t\t```\n\t\t\t\n\t\t{prompt}\n\t\t\t```\n\t\t\tHere:\n\t\t\t- "{prompt}" is the LLM 
prompt\n\n\t\t\tModel can produce erroneous output if a prompt is not well defined. \t\t\tIn our collaboration, 
we’ll work together to refine a prompt. The process consists of two main steps:\n\n\t\t\t## Step 1\n\t\t\tI will 
provide you with the current prompt along with prediction examples. Each example contains the input text, the final
prediction produced by the model, and the user feedback. \t\t\tUser feedback indicates whether the model prediction
is correct or not. \t\t\tYour task is to analyze the examples and user feedback, determining whether the 
\t\t\texisting instruction is describing the task reflected by these examples precisely, and suggests changes to 
the prompt to address the incorrect predictions.\n\n\t\t\t## Step 2\n\t\t\tNext, you will carefully review your 
reasoning in step 1, integrate the insights to refine the prompt, \t\t\tand provide me with the new prompt that 
improves the model’s performance.\n\t\t\t## Current prompt\n\t\t\t"Carefully read and understand the given 
mathematical problem. Make sure to correctly interpret all the details provided in the problem. Break it down into 
manageable steps, ensuring each step is correctly executed. Perform the necessary calculations with precision, and 
provide the final numerical answer. Double-check your calculations to ensure accuracy. If the problem involves 
multiple steps or calculations, ensure each step is correctly executed to arrive at the correct final 
answer."\n\t\t\t## Examples\n\t\t\t### Example #0\n\nQ: Reese had a sum of money in her savings account. The 
company she used to work with was not doing well which is why she lost her job. So she spent 20% of her savings for
her expenses in February, 40% of it in March, and $1500 in April. How much did she have in her savings account if 
she still has $2900 after spending those amounts?\n\nThe final numerical answer: nan\n\nUser feedback: nan\n\n\n###
Example #1\n\nQ: Laura needs to buy window treatments for 3 windows in her house.  She will need to buy a set of 
sheers and a set of drapes for each window.  The sheers cost $40.00 a pair and the drapes cost $60.00 a pair.  How 
much will the window treatments cost?\n\nThe final numerical answer: $300.00\n\nUser feedback: Incorrect. Must be 
equal to She needs a set of sheers that cost $40.00 and a set of drapes that cost $60.00 per window so that will 
cost her 40+60 = $<<40+60=100.00>>100.00\nShe had 3 windows she needs window treatments for they cost her $100.00 
each so 3*100 = $<<3*100=300.00>>300.00\n#### 300\n\n\n### Example #2\n\nQ: Elyse had 100 pieces of gum. She gave 
half to her brother Rick. He gave half of his to his friend Shane. Shane chewed 11 pieces of gum. How many pieces 
does Shane have left?\n\nThe final numerical answer: 39\n\nUser feedback: Incorrect. Must be equal to Elyse gave 
half of her 100 pieces of gum, or 100 / 2 = <<100/2=50>>50 pieces of gum, to Rick.\nRick gave half of his 50 pieces
of gum, or 50 / 2 = <<50/2=25>>25 pieces of gum, to Shane.\nShane had 25 pieces of gum and chewed 11, so he had 25 
– 11 = <<25-11=14>>14 pieces of gum left.\n#### 14\n\n\n### Example #3\n\nQ: In a store, there are three types of 
cheese: white, swiss, and blue cheese. Each cheese is packaged, and there are 200 grams of cheese in each package. 
The package prices are $5 for white cheese, $6 for swiss cheese, and $8 for blue cheese. Miley needed to get some 
for her party, and she decided to buy 5 packs of swiss cheese, 600 grams of blue cheese, and one-third less white 
che

{'input': '\n\t\t\tNow please carefully review your reasoning in Step 1 and help with Step 2: refining the 
prompt.\n\t\t\t## Current prompt\n\t\t\t"Carefully read and understand the given mathematical problem. Make sure to
correctly interpret all the details provided in the problem. Break it down into manageable steps, ensuring each 
step is correctly executed. Perform the necessary calculations with precision, and provide the final numerical 
answer. Double-check your calculations to ensure accuracy. If the problem involves multiple steps or calculations, 
ensure each step is correctly executed to arrive at the correct final answer."\n\n\t\t\t## Follow this guidance to 
refine the prompt:\n\n\t\t\t1. The new prompt should should describe the task precisely, and address the points 
raised in the user feedback.\n\n\t\t\t2. The new prompt should be similar to the current instruction, and only 
differ in the parts that address the issues you identified in Step 1.\n\t\t\t\t\tExample:\n\t\t\t\t\t- Current 
prompt: "The model should generate a summary of the input text."\n\t\t\t\t\t- New prompt: "The model should 
generate a summary of the input text. Pay attention to the original style."\n\n\t\t\t3. Reply only with the new 
prompt. Do not include input and output templates in the prompt. remove any special characters from the prompt.'}

reasoning: The current prompt instructs the model to carefully read and understand the given mathematical problem, 
break it down into manageable steps, perform the necessary calculations with precision, and provide the final 
numerical answer. However, the model seems to struggle with problems that involve multiple steps or require a 
certain level of interpretation. 

In Example #1, the model failed to understand that the cost of window treatments for each window includes both 
sheers and drapes. In Example #2, the model didn't correctly follow the sequence of actions (Elyse giving gum to 
Rick, Rick giving gum to Shane, and Shane chewing some gum). In Example #4, the model didn't correctly calculate 
the total number of items and the number of crates needed.

The prompt could be improved by explicitly instructing the model to consider all elements of the problem, to follow
the sequence of actions or calculations, and to double-check its calculations. The prompt could also remind the 
model to consider the units involved (e.g., dollars, pieces of gum, crates) and to ensure that its answer is in the
correct unit.

Here's a suggested new prompt:

"Carefully read and understand the given mathematical problem. Make sure to correctly interpret all the details 
provided in the problem, including the units involved. Follow the sequence

New prompt: "Carefully read and understand the given mathematical problem. Make sure to correctly interpret all the
details provided in the problem, including the sequence of actions or calculations and the units involved. Break it
down into manageable steps, ensuring each step is correctly executed. Perform the necessary calculations with 
precision, and provide the final numerical answer in the correct unit. Double-check your calculations to ensure 
accuracy. If the problem involves multiple steps or calculations, ensure each step is correctly executed to arrive 
at the correct final answer."

Train is done!

In [12]:
from ally.utils.logs import print_text


print_text(agent.skills['math_solver'].instruction_template)

"Carefully read and understand the given mathematical problem. Make sure to correctly interpret all the details 
provided in the problem, including the sequence of actions or calculations and the units involved. Break it down 
into manageable steps, ensuring each step is correctly executed. Perform the necessary calculations with precision,
and provide the final numerical answer in the correct unit. Double-check your calculations to ensure accuracy. If 
the problem involves multiple steps or calculations, ensure each step is correctly executed to arrive at the 
correct final answer."

In [14]:
result_new = agent.run(df_test.drop(columns='answer')[:10])
accuracy = StaticEnvironment(df=df_test, matching_function=evaluate_answers).get_feedback(skills, result_new).get_accuracy()
print(f'New accuracy: {accuracy["answer"]}')

Applying skill: math_solver

  0%|          | 0/10 [00:00<?, ?it/s]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for 
her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. 
How much in dollars does she make every day at the farmers' market?"}

 20%|██        | 2/10 [00:01<00:05,  1.51it/s]

{'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it 
take?'}

...

{'question': 'John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot 
something very important at home.  He tries to get home in 4 hours but spends the first 2 hours in standstill 
traffic.  He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time 
of the 4 hours going at 80 mph.  How far is he from home at the end of those 4 hours?'}

100%|██████████| 10/10 [00:34<00:00,  4.60s/it]

{'question': "Eliza's rate per hour for the first 40 hours she works each week is $10. She also receives an 
overtime pay of 1.2 times her regular hourly rate. If Eliza worked for 45 hours this week, how much are her 
earnings for this week?"}

100%|██████████| 10/10 [00:40<00:00,  4.04s/it]

New accuracy: 0.2857142857142857
